# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [9]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

2025-10-28 12:37:13,324 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [10]:
# TWrite a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
You are a Database Administrator (DBA). 

Based on the following markdown PRD content, generate a SQL schema 
for a relational database. 

<prd_content>
{prd_content}
</prd_content>

Design a normalized SQL schema with at least two tables (e.g.: users and
onboarding_tasks):
- The 'users' table should include an id, name, email, and role
(e.g.: 'New Hire', 'Manager').
- The 'onboarding_tasks' table should include an id, a title,
a description, a due_date, status, (e.g.: 'Pending', 'Completed'), and
a user_id foreign key.

 Include appropriate fields, data types, primary keys,
and foreign keys. Ensure the schema supports efficient querying and data 
integrity. Provide the SQL schema in a code block without any 
additional explanation.

**OUTPUT REQUIREMENTS**:
- Output should be the raw CREATE TABLE statements in a single 
SQL code block.
- Save the generated SQL to artifacts/schema.sql.
"""

print("--- Generating SQL Schema ---")
if prd_content:
    enhanced_schema_prompt = prompt_enhancer(schema_prompt)
    print("Schema Enhanced prompt\n", enhanced_schema_prompt)
    generated_schema = get_completion(enhanced_schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/schema.sql', overwrite=True)
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---


2025-10-28 12:37:17,140 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


Schema Enhanced prompt
 <persona>
You are a Senior Database Architect with extensive experience designing highly-normalized, production-grade relational schemas.
</persona>

<context>
Below is the complete Product Requirements Document (PRD) for “WelcomePath: New Hire Onboarding Platform.”  
Use it as the authoritative source for functional scope and data entities.

--- PRD START ---
# Product Requirements Document: WelcomePath: New Hire Onboarding Platform
| Status | **Draft** |
| **Author** | Product Team Alpha |
| **Version** | 1.0 |
| **Last Updated** | 2023-10-27 |
[ full PRD content exactly as supplied by the user ]
--- PRD END ---
</context>

<instructions>
1. Think step by step to identify every core entity, relationship, and attribute required to satisfy the PRD’s Version 1.0 scope (Stories 1.1, 2.1, 3.1).  
2. Apply 3rd-Normal-Form design principles.  
3. Create a minimum of two tables—users and onboarding_tasks—then add any additional tables or junction tables necessary for 

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [11]:
# TWrite a prompt to generate realistic seed data.
seed_data_prompt = f"""
You are a data specialist. Based on the PRD and cleaned SQL 
schema provided, generate 5 to 10 realistic INSERT statements to populate 
the database tables with data relevant to a new hire onboarding tool 
(e.g., sample user names and tasks like "Complete HR Paperwork").

**PRD CONTENT**:
<prd_content>
{prd_content}
</prd_content> 

**SQL SCHEMA**:
<cleaned_schema>
{cleaned_schema}
</cleaned_schema>

**OUTPUT REQUIREMENTS**:
- Output only the raw 'SQL' INSERT statements.
- Save the generated INSERT statements to artifacts/seed_data.sql.
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    enhanced_seed_data_prompt = prompt_enhancer(seed_data_prompt)
    print("Seed Data Enhanced prompt\n", enhanced_seed_data_prompt)
    generated_seed_data = get_completion(enhanced_seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql', overwrite=True)
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---


2025-10-28 12:38:39,994 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


Seed Data Enhanced prompt
 <prompt>
  <persona>
    You are a Senior Data Engineer who specializes in seeding relational databases for green-field SaaS applications. You have deep knowledge of SQL, foreign-key integrity, and realistic data modeling for HR and onboarding platforms.
  </persona>

  <context>
    Product: “WelcomePath” — a new-hire onboarding web application.  
    Goal: Seed the development database with believable starter data that matches both the PRD vision (digital paperwork, task checklists, role-based templates, etc.) and the SQL schema below.

    ---  PRD Highlights  ---
    • New-hire personas: Alex Chen (Engineer), etc.  
    • Core tasks: “Complete HR Paperwork”, “Schedule 1:1 with Manager”, “Meet Your Team”, etc.  
    • Roles: New Hire, Hiring Manager, HR Admin, IT.  
    • Mandatory tasks should default to TRUE; status should start as ‘pending’.  
    • Dates should be plausible (e.g., start_date within the next 30 days).

    ---  SQL Schema (abridged)  --

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [13]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files.

    This function is defensive: it validates that the schema/seed files
    contain SQL-like content before attempting to execute them. It uses
    executescript() for multi-statement SQL and always closes the DB
    connection in the finally block.
    """
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return

    # Delete database file if it exists to start fresh
    if os.path.exists(db_path):
        try:
            os.remove(db_path)
            print(f"Removed existing database file at {db_path}")
        except Exception as e:
            print(f"Warning: could not remove existing db file: {e}")

    conn = None
    try:
        # Connect (creates file if missing)
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # Read the schema SQL from disk
        try:
            with open(schema_path, "r", encoding="utf-8") as f:
                schema_sql = f.read()
        except Exception as e:
            print(f"Error reading schema file: {e}")
            return

        if not schema_sql.strip():
            print(f"Warning: Schema file {schema_path} is empty. No tables created.")
            return

        # Quick sanity check: ensure the content looks like SQL (contains CREATE TABLE)
        if "create table" not in schema_sql.lower():
            preview = schema_sql.strip()[:200].replace("\n", " ")
            print("Schema file does not look like valid SQL. Aborting execution.")
            print("Schema preview:", preview)
            return

        # Execute the schema SQL (multi-statement)
        cursor.executescript(schema_sql)
        print("Tables created successfully.")

        # If seed file exists, read and execute it
        if os.path.exists(seed_path):
            try:
                with open(seed_path, "r", encoding="utf-8") as f:
                    seed_sql = f.read()
            except Exception as e:
                print(f"Error reading seed file: {e}")
                seed_sql = ""

            if seed_sql.strip():
                # Use executescript to allow multiple INSERT statements
                cursor.executescript(seed_sql)
                print("Seed data executed successfully.")
            else:
                print(f"Warning: Seed file {seed_path} is empty. No seed data applied.")
        else:
            print(f"No seed file found at {seed_path}; skipping seed step.")

        # Commit the changes
        conn.commit()
        print("Database changes committed.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
            print("Rolled back changes due to error.")
    finally:
        # Ensure the connection is closed
        if conn:
            conn.close()
            print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

Removed existing database file at c:\Users\labadmin\Documents\AG-AISOFTDEV\artifacts\onboarding.db
Successfully connected to database at c:\Users\labadmin\Documents\AG-AISOFTDEV\artifacts\onboarding.db
Tables created successfully.
Seed data executed successfully.
Database changes committed.
Database connection closed.


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.